In [ ]:
# We establish the channel width & upstream position on the grid

channel_w = 20. # set channel width, meters
channel_center = size_x/(2 * spacing)
print(channel_center)

channel_diff = channel_w/2

num_nodes = size_x * size_y / spacing - 1
node_channel_left = int(num_nodes - channel_center - channel_diff)-size_x
node_channel_right = int(num_nodes - channel_center + channel_diff)-size_x

In [ ]:
# We set fixed boundary conditions, specifying the nodes and links in which the water is flowing into the grid
inlet_nodes = np.arange(math.floor(node_channel_left), math.ceil(node_channel_right), spacing).astype(int)

# We set the fixed values in the entry nodes/links
constant_Q = 100. # discharge value; m3/s
Q_array = np.ones(len(inlet_nodes)) * constant_Q

# We establish the initial conditions for depth (empty)
h = grid.add_zeros("surface_water__depth", at="node", clobber=True)

# Water velocity is zero in everywhere since there is no water yet
vel = grid.add_zeros("surface_water__velocity", at="link", clobber=True)

# Calculating the initial water surface elevation from water depth and topographic elevation
wse = grid.add_field("surface_water__elevation", Z, at="node", clobber=True)

# We set some other values
mannings_n = 0.05 # Mannings roughness
dt = 10 # Timestep; this is probably defined above/elsewhere

# Instantiate the OverlandFlow component to work on this grid and run it
of = OverlandFlow(grid, steep_slopes=True)

# Give the downward component of each inlet node a discharge corresponding to constant_Q
for n in inlet_nodes:
    links = grid.links_at_node[n]  #links pointing dow
    grid.at_link["surface_water__discharge"][links[3]] = constant_Q

# Iterate over time; run the OverlandFlow and refresh the discharge at the inlet
for i in range(100):
    of.run_one_step()
    for n in inlet_nodes:
        links = grid.links_at_node[n]  #links pointing down
        grid.at_link["surface_water__discharge"][links[3]] = constant_Q

grid.imshow('surface_water__depth',cmap="Blues")